# LSTM/GRU/MLP univariate/multivariate

### Create 2d array

In [ ]:
import numpy as np
import pandas as pd

# Choose frequency, binsize, longitude, latitude
freq = 'M'
binsize = 10
longitude_W = 134 # minimum is 134
longitude_E = 174 # maximum is 174
latitude_S = 10 # minimum is 10
latitude_N = 60 # minimum is 60

# load earthquake data for defined area
data = pd.read_csv('data/Japan_10_60_134_174_1973_2023_V2.csv', index_col=0)
data['Time'] = pd.to_datetime(data.Time)
data = data[(data.Longitude >= longitude_W) & (data.Longitude <= longitude_E) & (data.Latitude >= latitude_S) & (data.Latitude <= latitude_N)]
data.set_index('Time', inplace=True)
df = data.sort_index()

# Bin the longitude and latitude values into 2x2 degree bins
df['Longitude_bin'] = pd.cut(df['Longitude'], bins=np.arange(longitude_W, longitude_E+1, binsize))  # Change bin size to 2 degrees
df['Latitude_bin'] = pd.cut(df['Latitude'], bins=np.arange(latitude_S, latitude_N+1, binsize))  # Change bin size to 2 degrees

# Group the data by longitude bin, latitude bin, depth bin, and day, and compute the maximum magnitude within each group
grouped = df.groupby(['Longitude_bin', 'Latitude_bin', pd.Grouper(freq=freq, level="Time")]).max()['Magnitude']
grouped = grouped.unstack().fillna(0)

# Reshape the resulting data into a tensor with shape (1, time, depth, longitude, latitude)
time = len(grouped.columns)
longitude = len(grouped.index.levels[0])
latitude = len(grouped.index.levels[1])
tensor = np.zeros((time, longitude, latitude))

for t in range(time):
    tensor[t, :, :] = grouped.iloc[:, t].values.reshape(longitude, latitude)

# Rotate dimensions corresponding to 20 and 25, 90 degrees anti-clockwise
tensor = np.transpose(tensor, axes=(0, 2, 1))
tensor = np.flip(tensor, axis=1)

# Print the shape of the resulting tensor
print(tensor.shape)

# Reshape tensor into matrix
matrix = np.reshape(tensor, (tensor.shape[0], -1))

# Keep only the columns with at least one number bigger than 0
matrix = matrix[:, (matrix > 0).sum(axis=0) >= 0]
balance = 1 - (matrix >= 4.5).flatten().sum() / len(matrix.flatten())
print(matrix.shape, "mean:", matrix[matrix > 0].mean(), balance)

### Select most active sub-region

In [ ]:
df_transformed = pd.DataFrame(matrix).set_index(grouped.T.index)

active_column = df_transformed.sum(axis=0).argmax()
active_sub = df_transformed[active_column]
active_vec = np.reshape(np.array(active_sub), (-1, 1))
# print column number with highest sum
print("column with highest sum of magnitudes:", active_column)

In [ ]:
import seaborn as sns
import numpy as np

sns.set(rc={'figure.figsize':(20,3)})
plot = sns.lineplot(data=df_transformed[df_transformed >= 0], linewidth = .2, legend=False)
plot.set_xlabel("Time in months")
plot.set_ylabel("Magnitude")
plot.set_title("Earthquakes for selected sub-regions")

### Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split

# split data in train, val, test

# Activate when only forecasting one region
# matrix = np.reshape(matrix, (matrix.shape[0], matrix.shape[1], 1))

train, val_test = train_test_split(matrix, test_size=.2, shuffle=False, random_state=43)
val, test = train_test_split(val_test, test_size=.5, shuffle=False, random_state=43)

### Generate datasets from timeseries V1 (not shuffled)

In [ ]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
import tensorflow as tf

def dataset_generator(data, seq_length, cutoff):

  input_data = data # data[:-seq_length]
  targets = data[seq_length:]
  dataset = timeseries_dataset_from_array(input_data, (targets >= cutoff).astype(int), sequence_length=seq_length, sampling_rate=1, sequence_stride=1, shuffle=False, batch_size=len(data))
  """
  for batch in dataset:
    inputs, targets = batch
    assert np.array_equal(inputs[0], data[:seq_length])  # First sequence: steps [0-9]
    assert np.array_equal(targets[0], data[seq_length])  # Corresponding target: step 10
    """
  return dataset

# Set lookback timewindow
timewindow = 12
cutoff = 6

train_dataset = dataset_generator(train, timewindow, cutoff)
val_dataset = dataset_generator(val, timewindow, cutoff)
test_dataset = dataset_generator(test, timewindow, cutoff)

# Create train set
for batch in train_dataset:
    X_train, y_train = batch

# y_train = tf.reshape(y_train, shape=[y_train.shape[0], 1, y_train.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_train = tf.cast(tf.reduce_max(y_train, axis=2, keepdims=True) > 0, dtype=tf.int32)

# Create validation set
for batch in val_dataset:
    X_val, y_val = batch

# y_val = tf.reshape(y_val, shape=[y_val.shape[0], 1, y_val.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_val = tf.cast(tf.reduce_max(y_val, axis=2, keepdims=True) > 0, dtype=tf.int32)

# Create test set
for batch in test_dataset:
    X_test, y_test = batch

# y_test = tf.reshape(y_test, shape=[y_test.shape[0], 1, y_test.shape[1]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
# y_test = tf.cast(tf.reduce_max(y_test, axis=2, keepdims=True) > 0, dtype=tf.int32)

################################# Use for MLP
# Flatten 1 and 2 dimensions of X's for multivariate MLP
# X_train = np.reshape(X_train, (X_train.shape[0],-1))
# X_val = np.reshape(X_val, (X_val.shape[0],-1))
# X_test = np.reshape(X_test, (X_test.shape[0],-1))

X_train.shape, y_train.shape

## Construct LSTM (univariate/multivariate)

In [480]:
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Flatten, Input, TimeDistributed, Dropout, RepeatVector, BatchNormalization
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np
import keras
from keras.callbacks import EarlyStopping
from keras import regularizers
from keras import backend as K

keras.backend.clear_session()

# define model
model = Sequential()

model.add(BatchNormalization())
model.add(LSTM(24, activation='relu',
               return_sequences=True,
               input_shape=(X_train.shape[2:]),
               kernel_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4),
               bias_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4),
               dropout=0.4
               ))

model.add(BatchNormalization())
model.add(LSTM(24, activation='relu',
               return_sequences=False,
               kernel_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4),
               bias_regularizer=regularizers.L1L2(l1=1e-4, l2=1e-4),
               dropout=0.4
               ))
model.add(BatchNormalization())

"""
model.add(Dense(12, input_dim=(X_train.shape[1]), activation='relu'))
model.add(BatchNormalization())
model.add(Dense(12, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(12, activation='relu'))
model.add(BatchNormalization())
"""

model.add(Dense(24, activation='relu'))
model.add(Dense(24, activation='relu'))
model.add(Dense(20, activation='sigmoid'))

opt = keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.BinaryFocalCrossentropy( apply_class_balancing=True, alpha=balance, gamma=3)
model.compile(optimizer=opt, loss=loss, metrics=[keras.metrics.Precision(), keras.metrics.Recall()])



### fit model

In [481]:
# early stopping
callback = EarlyStopping(monitor='val_loss', patience=5)
# fit model
history = model.fit(x=X_train,
                    y=y_train,
                    validation_data=(X_val,y_val),
                    steps_per_epoch=12,
                    batch_size=128,
                    epochs=10000,
                    verbose=1,
                    callbacks=[callback])

Epoch 1/10000
12/12 [==============================] - 2s 28ms/step - loss: 0.1170 - precision: 0.0474 - recall: 0.5493 - val_loss: 0.4188 - val_precision: 0.0521 - val_recall: 0.5556
Epoch 2/10000
12/12 [==============================] - 0s 8ms/step - loss: 0.1019 - precision: 0.0499 - recall: 0.4514 - val_loss: 0.2127 - val_precision: 0.0546 - val_recall: 0.4444
Epoch 3/10000
12/12 [==============================] - 0s 8ms/step - loss: 0.0930 - precision: 0.0576 - recall: 0.3696 - val_loss: 0.1140 - val_precision: 0.0620 - val_recall: 0.4167
Epoch 4/10000
12/12 [==============================] - 0s 8ms/step - loss: 0.0851 - precision: 0.0673 - recall: 0.3043 - val_loss: 0.0813 - val_precision: 0.0781 - val_recall: 0.4167
Epoch 5/10000
12/12 [==============================] - 0s 8ms/step - loss: 0.0776 - precision: 0.0830 - recall: 0.2587 - val_loss: 0.0685 - val_precision: 0.0839 - val_recall: 0.3611
Epoch 6/10000
12/12 [==============================] - 0s 7ms/step - loss: 0.0697 - 

### predict

In [478]:
y_pred = model.predict(X_test)

# y_pred = scaler.inverse_transform(y_pred)
# y_test = scaler.inverse_transform(test)

2/2 [==============================] - 0s 3ms/step


### evaluation metrics

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(np.array(y_test).flatten(), y_pred.flatten() >= .5)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(np.array(y_test).flatten(), y_pred.flatten() >= .5)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(np.array(y_test).flatten(), y_pred.flatten() >= .5)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(np.array(y_test).flatten(), y_pred.flatten() >= .5)
print('F1 score: %f' % f1)

class_names = ['M<4.5', 'M>=4.5']

print(classification_report(np.array(y_test).flatten(), y_pred.flatten() >= .5, target_names=class_names))

# Calculate the proportion of the majority per row, column combination over all batches
majority_prop = np.mean(train >= cutoff, axis=0)[:]

# Calculate the complement for values lower than 0.5
majority_prop = np.where(majority_prop < 0.5, 1 - majority_prop, majority_prop)
zeroR = majority_prop.mean()

print("zeroR:", round(zeroR,4))

### confusion matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
sns.set(rc={'figure.figsize':(5,5)})
p = sns.heatmap(confusion_matrix(np.array(y_test).flatten(), y_pred.flatten() >= .5), annot=True, fmt='g')
p.set_xlabel("Predicted")
p.set_ylabel("True")
p.xaxis.set_ticklabels(['M<6', 'M>=6'], ha="center", va="center")
p.yaxis.set_ticklabels(['M<6', 'M>=6'], rotation=0, va="center")

### ROC curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def plot_ROC_AUC(X,y):

    # Use the trained model to predict the class probabilities for the validation set
    y_prob = model.predict(X)
    # y_pred = scaler.inverse_transform(y_prob)
    # y_test = scaler.inverse_transform(y)

    # Compute micro-average ROC curve and ROC area
    fpr, tpr, _ = roc_curve(np.array(y_test).flatten(), y_pred.flatten())
    roc_auc = auc(fpr, tpr)

    # Plot micro-average ROC curve
    plt.figure(figsize=(5,5))
    plt.plot(fpr, tpr, label='ROC curve (area = {0:0.2f})'
            ''.format(roc_auc), color='blue', linewidth=2)

    plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Test)')
    plt.legend(loc="lower right")
    # plt.savefig(savefig)
    plt.show()

plot_ROC_AUC(X_test, y_test)

# ConvLSTM2D

### Create 5D tensor

In [ ]:
import numpy as np
import pandas as pd

# Choose frequency, binsize, longitude, latitude
freq = 'W'
binsize = 5
longitude_W = 134 # minimum is 134
longitude_E = 174 # maximum is 174
latitude_S = 10 # minimum is 10
latitude_N = 60 # minimum is 60

# load earthquake data for defined area
data = pd.read_csv('data/Japan_10_60_134_174_1973_2023_V2.csv')
data['Time'] = pd.to_datetime(data.Time)
data = data[(data.Longitude >= longitude_W) & (data.Longitude <= longitude_E) & (data.Latitude >= latitude_S) & (data.Latitude <= latitude_N)]
data.set_index('Time', inplace=True)
df = data.sort_index()

# Bin the longitude and latitude values into 2x2 degree bins
df['Longitude_bin'] = pd.cut(df['Longitude'], bins=np.arange(longitude_W, longitude_E+1, binsize))  # Change bin size to 2 degrees
df['Latitude_bin'] = pd.cut(df['Latitude'], bins=np.arange(latitude_S, latitude_N+1, binsize))  # Change bin size to 2 degrees

# Group the data by longitude bin, latitude bin, depth bin, and day, and compute the maximum magnitude within each group
grouped = df.groupby(['Longitude_bin', 'Latitude_bin', pd.Grouper(freq=freq, level="Time")]).max()['Magnitude']
grouped = grouped.unstack().fillna(0)

# Reshape the resulting data into a tensor_convLSTM with shape (1, time, depth, longitude, latitude)
time = len(grouped.columns)
longitude = len(grouped.index.levels[0])
latitude = len(grouped.index.levels[1])
tensor_convLSTM = np.zeros((1, time, longitude, latitude, 1))

for t in range(time):
    tensor_convLSTM[0, t, :, :, 0] = grouped.iloc[:, t].values.reshape(longitude, latitude)

# Rotate dimensions corresponding to 20 and 25, 90 degrees anti-clockwise
tensor_convLSTM = np.transpose(tensor_convLSTM, axes=(0, 1, 4, 3, 2))
tensor_convLSTM = np.flip(tensor_convLSTM, axis=3)
# Print the shape of the resulting tensor_convLSTM
print(tensor_convLSTM.shape)


### Plot timesteps of 5D tensor

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Choose a timestep to plot (e.g. the first timestep)
timestep = 40

# Extract the data for the chosen timestep from the tensor
# tensor_convLSTM = tf.cast(tf.reduce_max(tensor_convLSTM, axis=2, keepdims=True) > 0, dtype=tf.int32)

data = tensor_convLSTM[0, timestep, 0, :, :]

# Create a heatmap plot of the data using Seaborn
sns.set(rc={'figure.figsize':(4.8,6)})
sns.heatmap(data, cmap='viridis', vmin=-1, vmax=10, linewidths=0.5, linecolor='grey', annot=False)

# Set the plot title and axis labels
plt.title(f'Earthquake magnitudes at timestep {timestep}')
plt.xlabel('Longitude bin')
plt.ylabel('Latitude bin')

# Show the plot
plt.show()


### splitting the data

In [ ]:
from sklearn.model_selection import train_test_split

# split data in train en test set
dataset_convLSTM = tensor_convLSTM.reshape((tensor_convLSTM.shape[1], tensor_convLSTM.shape[2], tensor_convLSTM.shape[3], tensor_convLSTM.shape[4]))

train, val_test = train_test_split(dataset_convLSTM, test_size=.4, shuffle=False, random_state=43)
val, test = train_test_split(val_test, test_size=.5, shuffle=False, random_state=43)


### Generate datasets from timeseries V1 (not shuffled)

In [ ]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
import tensorflow as tf

def dataset_generator(data, seq_length, cutoff):

  input_data = data # data[:-seq_length]
  targets = data[seq_length:]
  dataset = timeseries_dataset_from_array(input_data, (targets >= cutoff).astype(int), sequence_length=seq_length, sampling_rate=1, sequence_stride=1, shuffle=False, batch_size=len(data))
  """
  for batch in dataset:
    inputs, targets = batch
    assert np.array_equal(inputs[0], data[:seq_length])  # First sequence: steps [0-9]
    assert np.array_equal(targets[0], data[seq_length])  # Corresponding target: step 10
    """
  return dataset

# Set lookback timewindow
timewindow = 10
cutoff = 4.5

train_dataset = dataset_generator(train, timewindow, cutoff)
val_dataset = dataset_generator(val, timewindow, cutoff)
test_dataset = dataset_generator(test, timewindow, cutoff)

# Create train set
for batch in train_dataset:
    X_train, y_train = batch

y_train = tf.reshape(y_train, shape=[y_train.shape[0], 1, y_train.shape[1], y_train.shape[2], y_train.shape[3]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
y_train = tf.cast(tf.reduce_max(y_train, axis=2, keepdims=True) > 0, dtype=tf.int32)

# Create validation set
for batch in val_dataset:
    X_val, y_val = batch

y_val = tf.reshape(y_val, shape=[y_val.shape[0], 1, y_val.shape[1], y_val.shape[2], y_val.shape[3]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
y_val = tf.cast(tf.reduce_max(y_val, axis=2, keepdims=True) > 0, dtype=tf.int32)

# Create test set
for batch in test_dataset:
    X_test, y_test = batch

y_test = tf.reshape(y_test, shape=[y_test.shape[0], 1, y_test.shape[1], y_test.shape[2], y_test.shape[3]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
y_test = tf.cast(tf.reduce_max(y_test, axis=2, keepdims=True) > 0, dtype=tf.int32)

In [ ]:
X_train.shape

### Generate datasets from timeseries V2 (shuffled)

In [ ]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
import tensorflow as tf

def dataset_generator(data, seq_length, cutoff):

  input_data = data # data[:-seq_length]
  targets = data[seq_length:]
  dataset = timeseries_dataset_from_array(input_data, (targets >= cutoff).astype(int), sequence_length=seq_length, sampling_rate=1, sequence_stride=1, shuffle=False, batch_size=len(data))
  """
  for batch in dataset:
    inputs, targets = batch
    assert np.array_equal(inputs[0], data[:seq_length])  # First sequence: steps [0-9]
    assert np.array_equal(targets[0], data[seq_length])  # Corresponding target: step 10
    """
  return dataset

timewindow = 10
cutoff = 4.5

batches = dataset_generator(dataset_convLSTM, timewindow, cutoff)
for batch in batches:
    X, y = batch
    
y = tf.reshape(y, shape=[y.shape[0], 1, y.shape[1], y.shape[2], y.shape[3]])
y = tf.cast(tf.reduce_max(y, axis=2, keepdims=True) > 0, dtype=tf.int32)

X_train, X_val_test = train_test_split(np.array(X), test_size=.4, shuffle=True, random_state=43)
X_val, X_test = train_test_split(X_val_test, test_size=.5, shuffle=True, random_state=43)

y_train, y_val_test = train_test_split(np.array(y), test_size=.4, shuffle=True, random_state=43)
y_val, y_test = train_test_split(y_val_test, test_size=.5, shuffle=True, random_state=43)

### Modelconstruction of convLSTM2D

In [ ]:
from keras import layers, regularizers
import keras
keras.backend.clear_session()

from tensorflow.keras import layers, models

# Construct the inputut layer with no definite frame size.
input = layers.Input(shape=(X_train.shape[1:]))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = layers.BatchNormalization()(input)
x = layers.ConvLSTM2D(
    filters=16,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
    data_format = "channels_first",
    # kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)
)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=1,
    kernel_size=(1, 1),
    padding="same",
    return_sequences=False,
    activation="relu",
    data_format = "channels_first",
    kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)
)(x)
x = layers.BatchNormalization()(x)

x = tf.reshape(x, (-1, 1, x.shape[1], x.shape[2], x.shape[3]))
x = layers.Conv3D(filters=x.shape[4], kernel_size=(3, 3, 3), activation="sigmoid", padding="same")(x)
# Next, we will build the complete model and compile it.
model = keras.models.Model(input, x)
print(model.summary())

model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=[keras.metrics.Precision(), keras.metrics.Recall(), 'accuracy'])

### Modeltraining of convLSTM2D

In [ ]:
# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=10)

# Define modifiable training hyperparameters.
epochs = 100
batch_size = 32

# Fit the model to the training data.
model.fit(x=X_train,
          y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1)

### Predict

In [ ]:
y_pred = model.predict(X_test)

### Plot predicted against true

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


# Choose timesteps to plot
timestep = 10

# Extract the data for the chosen timesteps from the tensor
data1 = y_pred[timestep, 0, 0, :, :] > .5
data2 = y_test[timestep, 0, 0, :, :]

# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 6))

# Plot the data in each subplot
sns.heatmap(data1, cmap='viridis', vmin=0, vmax=1, linewidths=0.5, linecolor='grey', annot=False, ax=ax1)
sns.heatmap(data2, cmap='viridis', vmin=0, vmax=1, linewidths=0.5, linecolor='grey', annot=False, ax=ax2)

# Set the plot titles and axis labels
ax1.set_title(f'Predicted {cutoff} magnitude at timestep {timestep}')
ax1.set_xlabel('Longitude bin')
ax1.set_ylabel('Latitude bin')

ax2.set_title(f'True {cutoff} magnitude at timestep {timestep}')
ax2.set_xlabel('Longitude bin')
ax2.set_ylabel('Latitude bin')

# Show the plot
plt.show()

### Confusion matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
p = sns.heatmap(confusion_matrix(np.array(y_test).flatten(), y_pred.flatten() >= 0.5), annot=True, fmt='g')
p.set_xlabel("Predicted")
p.set_ylabel("True")
p.xaxis.set_ticklabels(['M<4.5', 'M>=4.5'], ha="center", va="center")
p.yaxis.set_ticklabels(['M<4.5', 'M>=4.5'], rotation=0, va="center")

### Evaluation metrics

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(np.array(y_test).flatten(), y_pred.flatten() >= 0.5)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(np.array(y_test).flatten(), y_pred.flatten() >= 0.5)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(np.array(y_test).flatten(), y_pred.flatten() >= 0.5)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(np.array(y_test).flatten(), y_pred.flatten() >= 0.5)
print('F1 score: %f' % f1)

class_names = ['M<6', 'M>=6']

print(classification_report(np.array(y_test).flatten(), y_pred.flatten() >= 0.5, target_names=class_names))

# Calculate the proportion of the majority per row, column combination over all batches
majority_prop = np.mean(y_test, axis=0)

# Calculate the complement for values lower than 0.5
majority_prop = np.where(majority_prop < 0.5, 1 - majority_prop, majority_prop)
zeroR = majority_prop.mean()

print("zeroR:", round(zeroR,4))

### ROC curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def plot_ROC_AUC(X,y):

    # Use the trained model to predict the class probabilities for the validation set
    y_prob = model.predict(X)
    # y_pred = scaler.inverse_transform(y_prob)
    # y_test = scaler.inverse_transform(y)

    # Compute micro-average ROC curve and ROC area
    fpr, tpr, _ = roc_curve(np.array(y_test).flatten(), np.array(y_pred).flatten())
    roc_auc = auc(fpr, tpr)

    # Plot micro-average ROC curve
    plt.figure(figsize=(6,6))
    plt.plot(fpr, tpr, label='ROC curve (area = {0:0.2f})'
            ''.format(roc_auc), color='blue', linewidth=2)

    plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Test)')
    plt.legend(loc="lower right")
    # plt.savefig(savefig)
    plt.show()

plot_ROC_AUC(X_test, y_test)

# ConvLSTM3d

### Create 6D tensor

In [ ]:
import numpy as np
import pandas as pd

# Choose frequency, binsize, longitude, latitude
# Can be treated as 2D when depthsize is set to 700
freq = 'W'
binsize = 5
longitude_W = 134 # minimum is 134
longitude_E = 174 # maximum is 174
latitude_S = 10 # minimum is 10
latitude_N = 60 # minimum is 60
depthsize = 175

# load earthquake data for a specific area
data = pd.read_csv('data/Japan_10_60_134_174_1973_2023_V2.csv')
data['Time'] = pd.to_datetime(data.Time)
data = data[(data.Longitude >= longitude_W) & (data.Longitude <= longitude_E) & (data.Latitude >= latitude_S) & (data.Latitude <= latitude_N)]
data.set_index('Time', inplace=True)
df = data.sort_index()

# Bin the longitude and latitude values into 2x2 degree bins
df['Longitude_bin'] = pd.cut(df['Longitude'], bins=np.arange(longitude_W, longitude_E+1, binsize))  # Change bin size to 2 degrees
df['Latitude_bin'] = pd.cut(df['Latitude'], bins=np.arange(latitude_S, latitude_N+1, binsize))  # Change bin size to 2 degrees
df['Depth_bin'] = pd.cut(df['Depth'], bins=np.arange(0, 700+depthsize, depthsize))

# Group the data by longitude bin, latitude bin, depth bin, and day, and compute the maximum magnitude within each group
grouped = df.groupby(['Longitude_bin', 'Latitude_bin', 'Depth_bin', pd.Grouper(freq=freq, level="Time")]).max()['Magnitude']
grouped = grouped.unstack().fillna(0)

# Reshape the resulting data into a tensor6D_convLSTM3D with shape (1, time, depth, longitude, latitude)
time = len(grouped.columns)
depth = len(grouped.index.levels[2])
longitude = len(grouped.index.levels[0])
latitude = len(grouped.index.levels[1])
channels = 1
tensor6D_convLSTM3D = np.zeros((1, time, channels, longitude, latitude, depth))

for t in range(time):
    tensor6D_convLSTM3D[0, t, 0, :, :, :] = grouped.iloc[:, t].values.reshape(longitude, latitude, depth)

# Rotate dimensions corresponding to 20 and 25, 90 degrees anti-clockwise
tensor6D_convLSTM3D = np.transpose(tensor6D_convLSTM3D, axes=(0, 1, 2, 4, 3, 5))
tensor6D_convLSTM3D = np.flip(tensor6D_convLSTM3D, axis=3)
# Print the shape of the resulting tensor6D_convLSTM3D
print(tensor6D_convLSTM3D.shape)
print("mean:", tensor6D_convLSTM3D.flatten()[tensor6D_convLSTM3D.flatten() > 0].mean())

### Plot timesteps of 6D tensor

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Choose a timestep to plot (e.g. the first timestep)
timestep = 60
depth = 0
channel = 0

# Extract the data for the chosen timestep from the tensor
# tensor_convLSTM = tf.cast(tf.reduce_max(tensor_convLSTM, axis=2, keepdims=True) > 0, dtype=tf.int32)

data = tensor6D_convLSTM3D[0, timestep, channel, :, :, depth]

# Create a heatmap plot of the data using Seaborn
sns.set(rc={'figure.figsize':(4.8,6)})
sns.heatmap(data, cmap='viridis', vmin=-1, vmax=10, linewidths=0.5, linecolor='grey', annot=False)

# Set the plot title and axis labels
plt.title(f'Earthquake magnitudes at timestep {freq, timestep}')
plt.xlabel('Longitude bin')
plt.ylabel('Latitude bin')

# Show the plot
plt.show()

### Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split

# split data in train en test set
tensor6D_convLSTM3D = tensor6D_convLSTM3D.reshape((tensor6D_convLSTM3D.shape[1], tensor6D_convLSTM3D.shape[2], tensor6D_convLSTM3D.shape[3], tensor6D_convLSTM3D.shape[4], tensor6D_convLSTM3D.shape[5]))

train, val_test = train_test_split(tensor6D_convLSTM3D, test_size=.4, shuffle=False, random_state=43)
val, test = train_test_split(val_test, test_size=.5, shuffle=False, random_state=43)

### Generate dataset from timeseries V1 (not shuffled)

In [ ]:
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
import tensorflow as tf

def dataset_generator(data, seq_length, cutoff):

  input_data = data # data[:-seq_length]
  targets = data[seq_length:]
  targets = (targets >= cutoff).astype(int)
  dataset = timeseries_dataset_from_array(input_data, targets, sequence_length=seq_length, sampling_rate=1, sequence_stride=1, shuffle=False, batch_size=len(data))
  """
  for batch in dataset:
    inputs, targets = batch
    assert np.array_equal(inputs[0], data[:seq_length])  # First sequence: steps [0-9]
    assert np.array_equal(targets[0], data[seq_length])  # Corresponding target: step 10
    """
  return dataset

# Set lookback timewindow
timewindow = 52
cutoff = 4.5

train_dataset = dataset_generator(train, timewindow, cutoff)
val_dataset = dataset_generator(val, timewindow, cutoff)
test_dataset = dataset_generator(test, timewindow, cutoff)

# Create train set
for batch in train_dataset:
    X_train, y_train = batch

y_train = tf.reshape(y_train, shape=[y_train.shape[0], 1, 1, y_train.shape[2], y_train.shape[3], y_train.shape[4]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
y_train = tf.cast(tf.reduce_max(y_train, axis=5, keepdims=True) > 0, dtype=tf.int32)

# Create validation set
for batch in val_dataset:
    X_val, y_val = batch

y_val = tf.reshape(y_val, shape=[y_val.shape[0], 1, 1, y_val.shape[2], y_val.shape[3], y_val.shape[4]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
y_val = tf.cast(tf.reduce_max(y_val, axis=5, keepdims=True) > 0, dtype=tf.int32)

# Create test set
for batch in test_dataset:
    X_test, y_test = batch

y_test = tf.reshape(y_test, shape=[y_test.shape[0], 1, 1, y_test.shape[2], y_test.shape[3], y_test.shape[4]])

# Collapse the depth dimension and converts all non-zero values to 1 and zero values to 0
y_test = tf.cast(tf.reduce_max(y_test, axis=5, keepdims=True) > 0, dtype=tf.int32)

### Modelconstruction of ConvLSTM3D

In [ ]:
from keras import layers, regularizers
import keras
keras.backend.clear_session()

from tensorflow.keras import layers, models

# Construct the inputut layer with no definite frame size.
input = layers.Input(shape=(X_train.shape[1:]))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = layers.BatchNormalization()(input)
x = layers.ConvLSTM3D(
    filters=12,
    kernel_size=(3, 3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
    data_format = "channels_first",
    # kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)
)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM3D(
    filters=1,
    kernel_size=(3, 3, 3),
    padding="same",
    return_sequences=False,
    # return_state=True,
    activation="relu",
    data_format = "channels_first",
    kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4) #l1 reduces weights to zero, l2 reduces towards zero
)(x)
x = layers.BatchNormalization()(x)
# x = x[0]
x = tf.reshape(x, (-1, 1, 1, x.shape[2], x.shape[3], x.shape[4]))
x = layers.Conv3D(filters=1, kernel_size=(3, 3, 3), activation="linear", padding="same")(x)

# Next, we will build the complete model and compile it.
model = keras.models.Model(input, x)
print(model.summary())

model.compile(loss='MSE', optimizer=keras.optimizers.Adam(learning_rate=0.001), metrics=[keras.metrics.Precision(), keras.metrics.Recall(), 'accuracy'])

### Modeltraining of ConvLSTM3D

In [ ]:
# Define some callbacks to improve training.
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 500
batch_size = 32

# Fit the model to the training data.
model.fit(x=X_train,
          y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr],
    verbose=1)

### Predict

In [ ]:
y_pred = model.predict(X_test)

### Confusion Matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
p = sns.heatmap(confusion_matrix(np.array(y_test).flatten(), y_pred.flatten() >= 0.5), annot=True, fmt='g')
p.set_xlabel("Predicted")
p.set_ylabel("True")
p.xaxis.set_ticklabels(['M<4.5', 'M>=4.5'], ha="center", va="center")
p.yaxis.set_ticklabels(['M<4.5', 'M>=4.5'], rotation=0, va="center")

### Evaluation Metrics

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(np.array(y_test).flatten(), y_pred.flatten() >= 0.5)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(np.array(y_test).flatten(), y_pred.flatten() >= 0.5)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(np.array(y_test).flatten(), y_pred.flatten() >= 0.5)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(np.array(y_test).flatten(), y_pred.flatten() >= 0.5)
print('F1 score: %f' % f1)

class_names = ['M<6', 'M>=6']

print(classification_report(np.array(y_test).flatten(), y_pred.flatten() >= 0.5, target_names=class_names))

# Calculate the proportion of the majority per row, column combination over all batches
majority_prop = np.mean(y_test, axis=0)[0, 0, :, :, 0]

# Calculate the complement for values lower than 0.5
majority_prop = np.where(majority_prop < 0.5, 1 - majority_prop, majority_prop)
zeroR = majority_prop.mean()

print("ZeroR:", round(zeroR,4))

### ROC curve

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

def plot_ROC_AUC(X,y):

    # Use the trained model to predict the class probabilities for the validation set
    y_prob = model.predict(X)
    # y_pred = scaler.inverse_transform(y_prob)
    # y_test = scaler.inverse_transform(y)

    # Compute micro-average ROC curve and ROC area
    fpr, tpr, _ = roc_curve(np.array(y_test).flatten(), np.array(y_pred).flatten())
    roc_auc = auc(fpr, tpr)

    # Plot micro-average ROC curve
    plt.figure(figsize=(6,6))
    plt.plot(fpr, tpr, label='ROC curve (area = {0:0.2f})'
            ''.format(roc_auc), color='blue', linewidth=2)

    plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Test)')
    plt.legend(loc="lower right")
    # plt.savefig(savefig)
    plt.show()

plot_ROC_AUC(X_test, y_test)